In [37]:
# Step 1: Install Required Packages
!pip install google-generativeai python-dotenv

# Step 2: Import Libraries
import os
import sqlite3
from dotenv import load_dotenv
import google.generativeai as genai

In [39]:
# Step 3: Load API Key from .env File
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

# Step 4: Load Gemini Model
model = genai.GenerativeModel("models/gemini-1.5-pro-latest")

In [53]:
# Step 5: Get User Preferences
learning_language = input("Which language do you want to learn? ")
known_language = input("Which language do you already know? ")
proficiency_level = input("Your level? (Beginner / Intermediate / Advanced): ")

# Step 6: Generate Roleplay Scene
scene_prompt = f"""
Create a short, friendly roleplay for a {proficiency_level.lower()} learner.
They know {known_language} and are learning {learning_language}.
Use casual tone and include basic phrases in {learning_language}.
"""

scene = model.generate_content(scene_prompt)
print("\n Scene:\n", scene.text)

Which language do you want to learn?  hindi
Which language do you already know?  english
Your level? (Beginner / Intermediate / Advanced):  beginner



 Scene:
 ## Coffee Shop Chit-Chat

**Characters:**

* **You:** A beginner Hindi learner.
* **Anika:** A friendly barista who speaks Hindi and English.


**(Scene: You walk into a coffee shop. Anika is behind the counter.)**


**Anika:** Hi there! Welcome! 

**You:** Hi!  

**Anika:** What can I get for you?

**You:**  Um... ek coffee, kripya. (One coffee, please.)

**Anika:** (Smiling)  Bilkul! (Certainly!)  Do you want milk with that? Doodh chahiye?

**You:** Haan, kripya. (Yes, please.)

**Anika:**  Aur kuch? (Anything else?)

**You:**  Shayad ek cookie? (Maybe a cookie?)

**Anika:**  Good choice! We have chocolate chip and oatmeal.  

**You:** Chocolate chip, kripya.

**Anika:** Okay!  That will be 50 rupees.

**You:**  (Handing over the money)  Yeh lo. (Here you go.)

**Anika:**  Shukriya! (Thank you!) Your coffee and cookie will be ready in just a minute.  Aap kaise hain? (How are you?)

**You:**  Main theek hoon, shukriya. Aap kaise hain? (I am fine, thank you. How are you?)

**

In [57]:
# Step 7: Start Interactive Chat
mistake_log = []
print("\n Chat started! Type 'exit' to stop.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("\n Chat ended. Mistakes will be shown next.")
        break

    prompt = f"""
You are a {learning_language} language tutor.
Respond only in {learning_language}.
Correct mistakes gently and explain why.
If correct, encourage the user.
User's input: "{user_input}"
"""

    reply = model.generate_content(prompt).text.strip()
    print("Bot:", reply)

    if any(x in reply.lower() for x in ["correct", "should be", "instead"]):
        mistake_log.append({"user_input": user_input, "correction": reply})


 Chat started! Type 'exit' to stop.



You:  exit



 Chat ended. Mistakes will be shown next.


In [59]:
#  Step 8: Show Mistake Summary
print("\n Mistake Summary:")

if not mistake_log:
    print("No noticeable mistakes.")
else:
    for i, m in enumerate(mistake_log, 1):
        print(f"\n🔸 Mistake {i}")
        print("Input:", m["user_input"])
        print("Feedback:", m["correction"])


 Mistake Summary:
No noticeable mistakes.


In [61]:
# Step 9: Generate Review Summary
feedback_prompt = f"""
You are a {learning_language} tutor.
The student made the following mistakes: {[m['user_input'] for m in mistake_log]}
Give a short, friendly summary of what they should improve.
"""

review = model.generate_content(feedback_prompt).text.strip()
print("\n Review Summary:\n", review)


 Review Summary:
 Okay, let's have a look.  It seems you're struggling a bit with [mention the specific area, e.g., genders, verb conjugations, postpositions].  Don't worry, these are common areas to trip up on!  With a little more practice, you'll get the hang of it. Let's focus on [mention the specific issue within the area, e.g., masculine vs. feminine nouns, matching verb endings with subject pronouns, using the correct postposition for location].  We can do some exercises together to help solidify this.  Keep up the good work! 😊


In [63]:
# Step 10: Save Mistakes to Database
conn = sqlite3.connect("mistakes.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS mistakes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_input TEXT,
    correction TEXT
)
""")

for m in mistake_log:
    cursor.execute("INSERT INTO mistakes (user_input, correction) VALUES (?, ?)", (m["user_input"], m["correction"]))

conn.commit()
print(f" {len(mistake_log)} mistake(s) saved to the database.")

 0 mistake(s) saved to the database.


In [69]:
# Step 11: View Saved Mistakes from Database
cursor.execute("SELECT * FROM mistakes")
rows = cursor.fetchall()

if rows:
    print(" Saved Mistakes:")
    for r in rows:
        print(f"\nID: {r[0]} | Input: {r[1]} | Correction: {r[2]}")
else:
    print("No saved mistakes. You're crushing it ")

 Saved Mistakes:

ID: 1 | Input: hii i am gaurav | Correction: The sentence has a few mistakes. "hii" is informal English and should be "Hi" or "Hello".  "i" needs to be capitalized as "I".  "am" is the English word for "to be".  In German, "to be" is "sein", and conjugated for "I", it's "bin".  "Gaurav" is your name and remains unchanged. The correct sentence in German is: "Hallo, ich bin Gaurav."

ID: 2 | Input: hii i am gaurav | Correction: The sentence "hii i am gaurav" has a few issues when translating to Spanish.  "hii" is an informal English greeting, and "i am" translates to "soy" when stating your name.  Capitalization is also important for proper names. The correct Spanish translation is "Hola, soy Gaurav."

ID: 3 | Input: Yo es bien, gracias. Y tú? | Correction: The main issue is with "Yo es bien".  The correct way to say "I am well" is "Yo estoy bien".  "Es" comes from the verb "ser" (to be) which is used for permanent or inherent characteristics.  "Estoy" comes from the ve